<a href="https://colab.research.google.com/github/MaimoonaKhilji/Analysis-Report-in-Rapidminer/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install agh_vqis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 13.4 MB/s eta 0:00:00
  Created wheel for PIMS: filename=PIMS-0.6.1-py3-none-any.whl size=82615 sha256=8b5e4b060ac91f21e07937cac41c645863f54d1dd861eaf087fdbb144ff04648
  Stored in directory: /root/.cache/pip/wheels/cc/bf/3e/bfa77232d942f8244145f9c713b6b38f6ef04b6fb5c021c114
Successfully built PIMS
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
  Attempting uninstall: scikit-learn
    F

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


from agh_vqis import process_single_mm_file, VQIs
from pathlib import Path
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
import shutil
# from sklearn.tree import DecisionTreeClassifier
# import numpy as np
import os
import pandas as pd
import pickle


RECALC = False


def read_vqis(my_file):
    """
    :param my_file: The name of the file to be read.
    :return: The data frame obtained from reading the CSV file.

    This method reads a CSV file located in the specified directory path and with the given file name. It then
    processes the contents of the file, adds a 'Frame' column with the value of * the file name, and removes the file
    from the directory. Finally, it returns the resulting data frame.
    """
    process_single_mm_file(Path(my_file))
    my_df = pd.read_csv(
        f"VQIs_for_{my_file[my_file.rfind('/')+1:-3]}csv"
    )  # Load the CSV file
    my_df["Frame"] = my_file
    os.remove(f"VQIs_for_{my_file[my_file.rfind('/')+1:-3]}csv")  # Remove the file
    return my_df


datasets = {"raw": "src", "reference": "pvs"}
vqis = {}
for dataset in datasets:
    directory_path = datasets[dataset]  # Specify the directory path

    if RECALC:
        # file_list = [
        #     file
        #     for file in os.listdir(directory_path)
        #     if os.path.isfile(os.path.join(directory_path, file))
        # ]  # Create a list of files in the given directory

        file_list = []
        for root, dirs, files in os.walk(
            directory_path
        ):  # Create a list of files in the given directory
            for file in files:
                file_list.append(os.path.join(root, file))
        # print(file_list)
        # exit(0)

        df = read_vqis(file_list[0])

        for file in file_list[1:]:
            new_row = read_vqis(file)
            df = pd.concat(
                [df, new_row], axis=0
            )  # Add the new row to the existing DataFrame

        with open(f"VQIs_for_{dataset}.pkl", "wb") as f:
            pickle.dump(df, f)  # Writing to disk
    with open(f"VQIs_for_{dataset}.pkl", "rb") as f:
        vqis[dataset] = pickle.load(f)  # Reading from disk
    #Display the first five rows
    #print(vqis[dataset].head())
    print(vqis[dataset].describe())
y_raw = pd.DataFrame(
    {"Class": ["raw"] * vqis["raw"].shape[0]}
)  # Create a DataFrame with one column and rows with the 'raw' value
y_ref = pd.DataFrame(
    {"Class": ["reference"] * vqis["reference"].shape[0]}
)  # Create a DataFrame with one column and rows with the 'reference' value
x = pd.concat([vqis["raw"], vqis["reference"]], axis=0)
y = pd.concat([y_raw, y_ref], axis=0)
#del x["Frame"]
del x["Slice"]
print(x)
print(y)
print(y.describe())
print(y.value_counts())
print(type(y))
# y["Class"] = np.random.choice(["raw", "reference"], size=len(y))
# print(y.value_counts())
# print(y.describe())
# clf = DecisionTreeClassifier()
print("-----------------------------------------------------------------------")
print(x.iloc[:,1:])
clf = RandomForestClassifier()
clf.fit(x.iloc[:,1:], y)
print(clf.feature_importances_)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
clf.fit(x_train.iloc[:,1:], y_train)
print(clf.score(x_test.iloc[:,1:], y_test))

print("+++++++++++++++++++++++++++++++++++++")
#print(x.iloc[[5]])
df1=x_test['Frame']
del x_test["Frame"]

for i in range(0,len(x_test)):

    pred=clf.predict(x_test.iloc[[i]])
    print(pred)
    actual = y_test.iloc[[i]]
    print(actual)
    print("======================",df1.values[i])
    source = df1.values[i] # Assuming you want the file path from the 6th row
    # Construct the source and destination paths
    source_path = f"{source}"

    if pred != actual.values:
        if actual.values == "raw":
            shutil.copy(source_path, 'wrong prediction/raw/')
        else:
            shutil.copy(source_path, 'wrong prediction/reference/')

